In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_colwidth',None)
pd.set_option('display.float_format',lambda x: '%.4f' % x)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as pl
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.pyplot import figure
import matplotlib.pylab as pll
from collections import Counter
from datetime import datetime
import time
import random
import math
import os
import itertools
from matplotlib import ticker
import matplotlib.colors as colors
from pylab import rcParams

In [ ]:
slurm1 = pd.read_csv('SlurmPlot2.csv')
slurm1.tail(100)

In [ ]:
slurm1.head(2)

In [ ]:
slurm = slurm1[['UID','JobID','Account','Submittimestamp','Start','End','Eligibletimestamp','Starttimestamp','Endtimestamp','AllocNodes','AllocCPUS','TimelimitRaw', 'NNodes','Submit','State','ExitCode','QOS','QOSRAW','Partition','TRESAllocMem','TRESINMaxUsagemem','TotalCPUtime']]

In [ ]:
slurm = slurm[slurm["Starttimestamp"] != 'Unknown']
slurm = slurm[slurm["Endtimestamp"] != 'Unknown']
slurm['JobLen'] = slurm['Endtimestamp'].astype(float) - slurm['Starttimestamp'].astype(float)
slurm['Totalres'] = slurm['JobLen'] * slurm['NNodes']
slurm['JobID'] = slurm['JobID'].astype(str)

In [ ]:
discard = ["."]
waittime = slurm[~slurm.JobID.str.contains('|'.join(discard),regex = False)]
waittime['Jcount'] = 1
waittime['Waittime'] = waittime['Starttimestamp'].astype(float) - waittime['Submittimestamp'].astype(float)

In [ ]:
waittime['year'] = pd.DatetimeIndex(waittime['Submit']).year
waittime['month'] = pd.DatetimeIndex(waittime['Submit']).month
waittime['year'] = waittime['year'].apply(str)
waittime['month'] = waittime['month'].apply(str)
waittime['yearmonth'] = waittime['year'] + '-' + waittime['month']

In [ ]:
waittime.head()

In [ ]:
waittime = waittime.loc[waittime.JobLen >= 0]

In [ ]:
waittime.info()

In [ ]:
slurm2 = slurm1.copy()

In [ ]:
slurm2['JobID'] = slurm2['JobID'].apply(str)
def case1(value):
    if '.' in value:
        return value.split('.')[0]
    else:
        return value
slurm2['JobID'] = slurm2['JobID'].map(lambda x: case1(x))

In [ ]:
slurm2['JobID'] = slurm2['JobID'].infer_objects()

In [ ]:
slurm3 = slurm2.groupby('JobID', as_index=False).agg({'MaxRSS':'max'})

In [ ]:
slurm3.head(5)

In [ ]:
waittimee = pd.merge(waittime, slurm3, on=['JobID'])

In [ ]:
waittimee.head(500)

In [ ]:

waittimee = waittimee.dropna(subset=['MaxRSS'])

In [ ]:
maxxx = waittimee.MaxRSS.values.max()
meann = waittimee.MaxRSS.values.mean()
print(maxxx)
print(meann)

In [ ]:
smallerrss = []
smallerrss = np.linspace(0, meann, 10,endpoint=False)
smallerrss = [l.tolist() for l in smallerrss]
smallerrss1 = np.linspace(0, smallerrss[1], 15 ,endpoint=False)
smallerrss1 = [l.tolist() for l in smallerrss1]
smallerrss1 = [ '%.1f' % elem for elem in smallerrss1 ]
smallerrss1

In [ ]:
del smallerrss1[0]

In [ ]:
smallerrss1 = np.asarray(smallerrss1, dtype = float)
coun = 0
for idx,row in slurm33.iterrows():
    for index, elem in enumerate(smallerrss1):
        if slurm33.loc[idx,'MaxRSS'] <= smallerrss1[0]:
            slurm33.loc[idx,'Ms 0-{}'.format(smallerrss1[0])] = 1    
        else:
            slurm33.loc[idx,'Ms 0-{}'.format(smallerrss1[0])] = 0   
        prevele = smallerrss1[index - 1]      
        if slurm33.loc[idx,'MaxRSS'] > prevele and slurm33.loc[idx,'MaxRSS'] <= elem:
            slurm33.loc[idx,'Ms {}-{}'.format(prevele,elem)] = 1
        else:
            slurm33.loc[idx,'Ms {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
slurm33.head()

In [ ]:
slurm33 = slurm33.drop(['Ms 627.9-44.9'], axis=1)
colrs = slurm33.columns
colrs = colrs.tolist()
colrs

In [ ]:
crs = slurm33.groupby('Account', as_index=False)[
 'Ms 0-44.9',
 'Ms 44.9-89.7',
 'Ms 89.7-134.6',
 'Ms 134.6-179.4',
 'Ms 179.4-224.3',
 'Ms 224.3-269.1',
 'Ms 269.1-314.0',
 'Ms 314.0-358.8',
 'Ms 358.8-403.7',
 'Ms 403.7-448.5',
 'Ms 448.5-493.4',
 'Ms 493.4-538.2',
 'Ms 538.2-583.1',
 'Ms 583.1-627.9'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in crs.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        
        ax.set_title("number of jobs according to Max RSS for the Account {}".format(str(count)),size=18)
        width = .5 
        temp.plot(kind = 'bar')
        ax.set_xlabel('MaxRSS criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_xticklabels([
 '0-44',
 '44-89',
 '89-134',
 '134-179',
 '179-224',
 '224-269',
 '269-314',
 '314-358',
 '358-403',
 '403-448',
 '448-493',
 '493-538',
 '538-583',
 '583-627'],size = 18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("18")
        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_MaxRSS_sub'):
            os.makedirs('Histogram_MaxRSS_sub')
        fig.savefig('Histogram_MaxRSS_sub/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

In [ ]:
partsr = []
parts1r = []
parts2r = []
parts3r = []
parts4r = []
partsr = np.linspace(0, meann, 10,endpoint=False)
parts1r = np.linspace(meann, maxxx, 4)
partsr = [l.tolist() for l in partsr]
parts1r = [l.tolist() for l in parts1r]
parts2r.append(np.linspace(parts1r[0], parts1r[1], 20,endpoint=False))
parts3r.append(np.linspace(parts1r[1], parts1r[2], 10,endpoint=False))
parts4r.append(np.linspace(parts1r[2], parts1r[3], 10))
parts2r = [l2.tolist() for l2 in parts2r]
parts2r = list(np.concatenate(parts2r).flat)
parts3r = [l3.tolist() for l3 in parts3r]
parts3r = list(np.concatenate(parts3r).flat)
parts4r = [l4.tolist() for l4 in parts4r]
parts4r = list(np.concatenate(parts4r).flat)

In [ ]:
partsr = partsr + parts2r + parts3r + parts4r
del partsr[0]
partsr

In [ ]:
partsr = [ '%.2f' % elem for elem in partsr ]

In [ ]:
partsr = np.asarray(partsr, dtype = float)

In [ ]:
coun = 0
for idx,row in slurm3.iterrows():
    for index, elem in enumerate(partsr):
        if slurm3.loc[idx,'MaxRSS'] <= partsr[0]:
            slurm3.loc[idx,'M 0-{}'.format(partsr[0])] = 1    
        else:
            slurm3.loc[idx,'M 0-{}'.format(partsr[0])] = 0   
        prevele = partsr[index - 1]      
        if slurm3.loc[idx,'MaxRSS'] > prevele and slurm3.loc[idx,'MaxRSS'] <= elem:
            slurm3.loc[idx,'M {}-{}'.format(prevele,elem)] = 1
        else:
            slurm3.loc[idx,'M {}-{}'.format(prevele,elem)] = 0
    coun = coun + 1
    if coun % 10000 == 0:
        print(coun)

In [ ]:
slurm3.head()

In [ ]:
slurm3 = slurm3.drop(['M 1291493.68-672.76'], axis=1)
col = slurm3.columns
col = col.tolist()
col

In [ ]:
crs = slurm3.groupby('Account', as_index=False)[
 'M 0-672.76',
 'M 672.76-1345.53',
 'M 1345.53-2018.29',
 'M 2018.29-2691.06',
 'M 2691.06-3363.82',
 'M 3363.82-4036.58',
 'M 4036.58-4709.35',
 'M 4709.35-5382.11',
 'M 5382.11-6054.87',
 'M 6054.87-6727.64',
 'M 6727.64-28140.41',
 'M 28140.41-49553.17',
 'M 49553.17-70965.94',
 'M 70965.94-92378.71',
 'M 92378.71-113791.47',
 'M 113791.47-135204.24',
 'M 135204.24-156617.01',
 'M 156617.01-178029.78',
 'M 178029.78-199442.54',
 'M 199442.54-220855.31',
 'M 220855.31-242268.08',
 'M 242268.08-263680.85',
 'M 263680.85-285093.61',
 'M 285093.61-306506.38',
 'M 306506.38-327919.15',
 'M 327919.15-349331.92',
 'M 349331.92-370744.68',
 'M 370744.68-392157.45',
 'M 392157.45-413570.22',
 'M 413570.22-434982.99',
 'M 434982.99-477808.52',
 'M 477808.52-520634.05',
 'M 520634.05-563459.59',
 'M 563459.59-606285.12',
 'M 606285.12-649110.66',
 'M 649110.66-691936.19',
 'M 691936.19-734761.73',
 'M 734761.73-777587.26',
 'M 777587.26-820412.8',
 'M 820412.8-863238.33',
 'M 863238.33-910822.26',
 'M 910822.26-958406.19',
 'M 958406.19-1005990.12',
 'M 1005990.12-1053574.04',
 'M 1053574.04-1101157.97',
 'M 1101157.97-1148741.9',
 'M 1148741.9-1196325.83',
 'M 1196325.83-1243909.75',
 'M 1243909.75-1291493.68'].sum()

In [ ]:
coun = 0
rcParams['figure.figsize'] = 20, 10
for idx,row in crs.iterrows():
        count = row.Account
        temp = row.drop(labels = ['Account'])
        print(temp)
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to MaxRSS for the Account {}".format(str(count)),size=18)
        width = .5 
        temp.plot(kind = 'bar')
        ax.set_xlabel('MaxRSS criteria', fontsize=18)
        ax.set_ylabel('Number of jobs', fontsize=18)
        ax.set_xticklabels( [
 '0-672',
 '672-1345',
 '1345-2018',
 '2018-2691',
 '2691-3363',
 '3363-4036',
 '4036-4709',
 '4709-5382',
 '5382-6054',
 '6054-6727',
 '6727-28140',
 '28140-49553',
 '49553-70965',
 '70965-92378',
 '92378-113791',
 '113791-135204',
 '135204-156617',
 '156617-178029',
 '178029-199442',
 '199442-220855',
 '220855-242268',
 '242268-263680',
 '263680-285093',
 '285093-306506',
 '306506-327919',
 '327919-349331',
 '349331-370744',
 '370744-392157',
 '392157-413570',
 '413570-434982',
 '434982-477808',
 '477808-520634',
 '520634-563459',
 '563459-606285',
 '606285-649110',
 '649110-691936',
 '691936-734761',
 '734761-777587',
 '777587-820412',
 '820412-863238',
 '863238-910822',
 '910822-958406',
 '958406-1005990',
 '1005990-1053574',
 '1053574-1101157',
 '1101157-1148741',
 '1148741-1196325',
 '1196325-1243909',
 '1243909-1291493'],size = 16)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("16")
        plt.show()
        coun = coun + 1
        if not os.path.exists('Histogram_MaxRSS'):
            os.makedirs('Histogram_MaxRSS')
        fig.savefig('Histogram_MaxRSS/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)

## mean of MaxRSS according to job size

In [ ]:
waittimee_JS = waittimee.groupby('NNodes', as_index=False).agg({'MaxRSS':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 60, 10
waittimee_JS['NNodes'] = waittimee_JS['NNodes'].apply(str)
plots = sns.barplot(x ='NNodes',y = 'MaxRSS',data =  waittimee_JS)
plots.set_title('Mean Maximum RSS according to different job size',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
if not os.path.exists('RSS_Analysis'):
    os.makedirs('RSS_Analysis')
plt.savefig('RSS_Analysis/RSS_Analysis_JS.png')

# report purpose

In [ ]:
waittimee_JS1 = waittimee_JS.loc[waittimee_JS.NNodes <= 50]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 38, 10
waittimee_JS1['NNodes'] = waittimee_JS1['NNodes'].apply(str)
plots = sns.barplot(x ='NNodes',y = 'MaxRSS',data =  waittimee_JS1)
plots.set_title('Average MaxRSS according to different job sizes',size = 22)
for bar in plots.patches:
    plots.annotate(int(bar.get_height()),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Job size',size = 22)
plt.ylabel('MaxRSS in megabytes',size = 22)
if not os.path.exists('RSS_Analysis'):
    os.makedirs('RSS_Analysis')
plt.savefig('RSS_Analysis/RSS_Analysis_JS1.png')

In [ ]:
waittimee_JS2 = waittimee_JS.loc[waittimee_JS.NNodes > 50]

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 30, 10
waittimee_JS2['NNodes'] = waittimee_JS2['NNodes'].apply(str)
plots = sns.barplot(x ='NNodes',y = 'MaxRSS',data =  waittimee_JS2)
plots.set_title('Average MaxRSS according to different job sizes',size = 22)
for bar in plots.patches:
    plots.annotate(int(bar.get_height()),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=18, xytext=(0, 8),
                   textcoords='offset points',rotation = 20)
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Job size',size = 22)
plt.ylabel('MaxRSS in megabytes',size = 22)
if not os.path.exists('RSS_Analysis'):
    os.makedirs('RSS_Analysis')
plt.savefig('RSS_Analysis/RSS_Analysis_JS2.png')

## MaxRSS mean according to project

In [ ]:
waittimee_PJ = waittimee.groupby('Account', as_index=False).agg({'MaxRSS':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 80, 10
waittimee_PJ['Account'] = waittimee_PJ['Account'].apply(str)
plots = sns.barplot(x ='Account',y = 'MaxRSS',data =  waittimee_PJ)
plots.set_title('Mean Maximum RSS according to different projects',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
if not os.path.exists('RSS_Analysis'):
    os.makedirs('RSS_Analysis')
plt.savefig('RSS_Analysis/RSS_Analysis_PJ.png')

## MaxRSS mean according to QOS

In [ ]:
waittimee_QOS = waittimee.groupby('QOS', as_index=False).agg({'MaxRSS':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
waittimee_QOS['QOS'] = waittimee_QOS['QOS'].apply(str)
plots = sns.barplot(x ='QOS',y = 'MaxRSS',data =  waittimee_QOS)
plots.set_title('Mean Maximum RSS according to job QOS',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=16, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('QoS',size = 22)
plt.ylabel('MaxRSS in megabytes',size = 22)
if not os.path.exists('RSS_Analysis'):
    os.makedirs('RSS_Analysis')
plt.savefig('RSS_Analysis/RSS_Analysis_QOS.png')

## MaxRSS mean according to partition

In [ ]:
waittimee_rss = waittimee.groupby('Partition', as_index=False).agg({'MaxRSS':'mean','Jcount':'sum'})

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
waittimee_rss['Partition'] = waittimee_rss['Partition'].apply(str)
plots = sns.barplot(x ='Partition',y = 'MaxRSS',data =  waittimee_rss)
plots.set_title('Mean Maximum RSS according to Partitions',size = 22)
for bar in plots.patches:
    plots.annotate(format(bar.get_height(), '.2f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=16, xytext=(0, 8),
                   textcoords='offset points')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Partition',size = 22)
plt.ylabel('MaxRSS in megabytes',size = 22)
if not os.path.exists('RSS_Analysis'):
    os.makedirs('RSS_Analysis')
plt.savefig('RSS_Analysis/RSS_Analysis_part.png')

## Number of jobs according to MaxRSS every year

In [ ]:
def fun(row):
    if row['MaxRSS'] <= 500:
        val = 1
    else:
        val = 0
    return val
waittimee['upto500MB'] = waittimee.apply(fun,axis =1)
def fun1(row):
    if row['MaxRSS'] <= 1000 and row['MaxRSS'] > 500:
        val = 1
    else:
        val = 0
    return val
waittimee['uptooneGB'] = waittimee.apply(fun1,axis =1)
def fun2(row):
    if row['MaxRSS'] <= 10000 and row['MaxRSS'] > 1000:
        val = 1
    else:
        val = 0
    return val
waittimee['upto10GB'] = waittimee.apply(fun2,axis =1)

def fun3(row):
    if row['MaxRSS'] > 10000 and row['MaxRSS'] <= 50000:
        val = 1
    else:
        val = 0
    return val
waittimee['upto50GB'] = waittimee.apply(fun3,axis =1)
# def fun4(row):
#     if math.isnan(row.MaxRSS):
#         val = 1
#     else:
#         val = 0
#     return val
# waittimee['RSSNaN'] = waittimee.apply(fun4,axis =1)
def fun5(row):
    if row['MaxRSS'] > 50000:
        val = 1
    else:
        val = 0
    return val
waittimee['morethan50GB'] = waittimee.apply(fun5,axis =1)

In [ ]:
def function8(years):
    coun = 10000
    df_list = []
    df_list2 = []
    for i in years:
        temp = waittimee[waittimee.yearmonth.str.contains(i,regex = False)]
        df_list.append(temp)
    for j in df_list:
        temp1 = j.groupby(['month','year'], as_index=False).agg({'Jcount':'sum','upto500MB':'sum','uptooneGB':'sum','upto10GB':'sum','upto50GB':'sum','morethan50GB':'sum'})
        temp1['month'] = temp1['month'].apply(int)
        temp1 = temp1.sort_values('month')
        df_list2.append(temp1)
    for l in df_list2:
        count = 0
        num = l.month
        datetime_object = []
        month_name = []
        for m in num:
            datetime_object.append(datetime.strptime(str(m), "%m"))
        for d in datetime_object:
            month_name.append(d.strftime("%b"))
        fig,ax = plt.subplots()
        ax.grid(False)
        ax.set_title("Number of jobs according to RSS usage for the year {}".format(str(l.year[0])),size=22)
        width = .5 
        ax.bar(l.month, l.upto500MB, width, color='g',alpha = 0.5)
        ax.bar(l.month, l.uptooneGB, width,bottom =l.upto500MB, color='b',alpha = 0.5)
        ax.bar(l.month, l.upto10GB, width,bottom =l.upto500MB + l.uptooneGB, color='brown',alpha = 0.5)
        ax.bar(l.month, l.upto50GB, width,bottom =l.upto500MB + l.uptooneGB + l.upto10GB , color='k',alpha = 0.5)
        ax.bar(l.month, l.morethan50GB, width,bottom =l.upto500MB + l.uptooneGB + l.upto10GB + l.upto50GB, color='yellow',alpha = 0.5)
        fig.set_figheight(12)
        fig.set_figwidth(16)
        # for p in ax.patches:
        #     ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005),fontsize=10)
        ax.set_xlabel('Months', fontsize=16)
        ax.set_ylabel('Number of RSS usage jobs', fontsize=18)
        ax.set(xticks = l.month)
        ax.set_xticklabels(month_name, fontsize=18)
        for tick in ax.get_yticklabels():
            tick.set_fontsize("16")
        plt.subplots_adjust(left=0, bottom=0)
        plt.legend(["RSS upto 500 MB",'RSS from 500MB to 1 GB', "RSS upto 10 GB","RSS upto 50 GB",'RSS more than 50 GB'],prop={'size': 22})
        coun = coun + 1
        if not os.path.exists('RSS_Analysis'):
            os.makedirs('RSS_Analysis')
        fig.savefig('RSS_Analysis/{}.png'.format(str(coun)),bbox_inches='tight', dpi=100)
        plt.show()

In [ ]:
years = ['2019', '2020', '2021', '2022']
function8(years)

In [ ]:
# years = ['2019', '2020', '2021', '2022']
# function9(years)

In [ ]:
# years = ['2019', '2020', '2021', '2022']
# function10(years)